In [24]:
import os
import time
import re
import chromedriver_autoinstaller
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [25]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [26]:
for index, gu_name in tqdm(enumerate(gu_list)):
    fileName = f'{gu_name}.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("맛집명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "평점" + "|" + "업종" + "|" + "대표사진주소" + "|" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    # 크롬 버젼 확인 후 새로운 드라이버 설치해서 실행
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito") # 시크릿 모드
    chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
    chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
    chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
    chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

    try: # 크롬 드라이버
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

    # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
    # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
    driver.implicitly_wait(10) # 10초정도 멈추기
    
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 맛집')  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, "info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_element(By.CSS_SELECTOR, ('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                try: place_rating = store_info.select('.rating > .score > .num')[0].text
                except : place_rating = "0"
                place_sub_category = store_info.select('.head_item > .subcategory')[0].text
                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_elements(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_rating + "|" + place_sub_category + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

0it [00:00, ?it/s]/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_4340/2060592008.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)
/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_4340/2060592008.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)


어반플랜트 합정 
피오니 홍대점 
바다회사랑 
프릳츠 도화점 
소이연남 
바다회사랑 2호점 
을밀대 본점 
크레이지카츠 
테이스티버거 
마포진짜원조최대포 본점 
황금콩밭 
오레노라멘 합정본점 
역전회관 
망원동즉석우동전문돈까스 본점 
어글리베이커리 
1  of  [  1  ] 
진미식당 
미쁘동 
마포나루 도화본점 
몽중헌 공덕점 
평이담백뼈칼국수 
하이디라오 홍대지점 
감성타코 홍대점 
원조조박집 본관 
헤키 
곱 마포점 
마포양지설렁탕 
943킹스크로스 
스케줄 합정 
우동카덴 
카페공명 
2  of  [  1  ] 
연교 
카페 레이어드 연남점 
우리바다수산 
진짜파스타 
아소정 
모연 
오츠커피 마포점 
중화복춘 
소금집델리 망원 
연남토마 본점 
리치몬드과자점 성산본점 
우와 홍대본점 
진진 본관 
비파티세리 공덕점 
포비 합정점 
3  of  [  1  ] 
마포옥 
맛이차이나 
앤트러사이트 합정점 
청어람 망원점 
각시보쌈 
두껍삼 마포직영점 
진대감 마포점 
마포원조떡볶이 
코코로카라 
정돈 
앤트러사이트 서교점 
조선시대 
또보겠지떡볶이집 호호시스터점 
툭툭누들타이 
테일러커피 서교점 
4  of  [  1  ] 
무신사테라스 
감나무집기사식당 
카와카츠 
당인리책발전소 
산울림1992 
한강껍데기 
옥동식 서교점 
일일향 마포점 
최강금돈까스 
마녀주방 홍대점 
색동저고리 마포점 
경기떡집 
뉴오더클럽 
일등식당 
어메이징농카이 
5  of  [  1  ] 
카쿠시타 
합정고깃집 
개화기요정 
황곱 홍대점 
다이닝원 상암점 
굴다리식당 
칸다소바 홍대점 
이치류 홍대본점 
톤앤매너 
스파카나폴리 
램랜드 
커피리브레 연남점 
몰토베네 
아오이토리 
빈브라더스 합정점 
1  of  [  2  ] 
피자네버슬립스 합정본점 
카미야 
콜린 
와이키키마켓 
프리모바치오바치 홍대본점 
가미우동 
땡스오트 연남점 
락희옥 마포본점 
중화복춘 골드동교점 
냉장고 
부산집 
오랑지 
장수갈매기 본점 
합정옥 
1984 
2  of  [  2  ] 
합정광안

1it [1:02:08, 3728.02s/it]

End of Page
Arrow is Disabled
End of Crawl
목란 
만동제과 
카덴 연희본점 
신촌형제갈비 
한옥집김치찜 서대문본점 
락희안 명지대본점 
고미정 
라구식당 
앤트러사이트 연희점 
피터팬1978 본점 
맛있는집 
철길떡볶이 
연희동칼국수 본점 
고삼이 신촌점 
연경 연희점 
1  of  [  1  ] 
신촌황소곱창 신촌본점 
소신이쏘 
시오 연희본점 
수빈 
더파이홀 
어머니와아들 
콘하스 연희점 
부엉이돈가스 신촌점 
정육면체 
카라멘야 
벙커1 
꼬숑돈까스 
복성각 
또보겠지떡볶이집 신촌히어로점 
이화원 
2  of  [  1  ] 
사조미가 
편의방 
미스터서왕만두 
미도인 신촌 
가야가야 이대점 
미분당 신촌2호점 
가타쯔무리 
매뉴팩트커피 연희본점 
독수리다방 
녹원쌈밥 
월순철판동태찜 
스웨이커피스테이션 
옐로우보울 
그릭데이 이대점 
참맛집 
3  of  [  1  ] 
클로리스 신촌점 
신촌수제비 
달인꽈배기 
심야식당텐조 
하이드미플리즈 
신촌해물칼국수 
부탄츄 신촌점 
청송함흥냉면전문점 본관 
조은집 
현명식탁 
연희곰탕 
가장맛있는곱창 부추곱창 신촌점 
연어초밥 
롯지190 
마더린러베이글 
4  of  [  1  ] 
모래내설농탕 본점 
복성각 서대문점 
연희김밥 본점 
여우골 
야바이 
정통집 
타코몽 
르솔레이 
설레임삼겹살1989 
라이라이 
산천마루 
서대문족발 본점 
로덴드론 
아웃백스테이크하우스 신촌점 
기꾸스시 
5  of  [  1  ] 
엠브로돈까스 신촌점 
쿳사 
이빠네마그릴 
푸어링아웃 
금옥당 연희점 
술익는마을 
유닭스토리 신촌점 
뚜띠쿠치나 서대문점 
키친봄날 
보승회관 신촌점 
소담촌 홍제점 
충정각 
현대수산 
마녀주방 신촌점 
오향만두 
1  of  [  2  ] 
춘천집닭갈비막국수 
엄마손떡볶이 본점 
호밀밭 
연탄생고기집 
맛있는칼국수 본점 
포석정 
벽제갈비 신촌점 
군자대한곱창 신촌점 
에브리띵베이글 
타코벨 신촌점 
유야케도쿄 
이품 
헬로방방 남가좌충암점 
미네르바 


2it [2:00:40, 3601.28s/it]

...

End of Page
Arrow is Disabled
End of Crawl
또순이네 
코끼리베이글 
대한옥 
송죽장 
카페1953위드오드리 
당산오돌 본점 
진주집 
정인면옥 본점 
세상의모든아침 여의도점 
그믐족발 본점 
올드문래 
삼해집 영등포점 
영일분식 
화목순대국 
삼거리먼지막순대국 
1  of  [  1  ] 
하이디라오 영등포점 
가양칼국수 
골목오리집 
부일갈비 
3일한우국밥 
승남집 
마굿간생고기 영등포점 
원조호수삼계탕 본점 
허브족발 
당산옛날곱창 
대관원 
강강술래 당산점 
이한스시 
마호가니커피 여의도점 
갈빗 
2  of  [  1  ] 
양키통닭 
러스트베이커리 
이조보쌈집 
아루히 
하동관 국회의사당역직영점 
쉐프조 
바스버거 여의도점 
대한옥 2호점 
흑돈가 여의도직영점 
청수 
함흥냉면 
교대이층집 여의도점 
신길동별난아찌짬뽕 
능이버섯백숙 
동해도 본점 
3  of  [  1  ] 
63뷔페 파빌리온 
당산원조곱창 
원조 부안집 
월화고기 문래동본점 
양키스그릴 
여로집 본관 
스시나베 
차이나플레인 여의도점 
창고43 본점 
호텔707 
대문점 
오복순대국 
오월의종 타임스퀘어점 
소문난식당 
최우영수산 영등포점 
4  of  [  1  ] 
육영토종한우 
본보야지 
스몰 
징기스 영등포점 
브로트아트 
진가와 여의도점 
모던눌랑 여의도점 
더플레이트디저트 더현대서울점 
브루클린더버거조인트 여의도점 
시즌커피앤베이크 
더블유페스타 
일일향 여의도점 
도쿄등심 여의도점 
화담 
곱 문래점 
5  of  [  1  ] 
고봉삼계탕 
진진만두 여의도점 
뚜스뚜스베이커리카페 선유도점 
오목집 당산점 
마루 
영등포 카페드람브르 
영등포꼬들집 
세광양대창 여의도점 
카페솔트 
팜스프링스 
일미집 영등포점 
메이탄 영등포점 
동해반점 
커피리브레 영등포 타임스퀘어점 
호박집 
1  of  [  2  ] 
육갑식당 당산점 
공주칼국수 
영등포꽃삼 
동래정 영등포본점 
에베레스트 레스토랑 영등포점 
양평동장군집 양평동본점 
서궁 
하동관

21it [20:36:59, 3573.15s/it]

End of Page
Arrow is Disabled
End of Crawl
가산물갈비&백년불고기 
강강술래 시흥점 
동흥관 
도원 
개성손만두요리전문점 가산디지털점 
동대문털보네 
순댕이네얼큰수제비 
두껍삼 가산직영점 
북촌삼대갈비 가산본점 
인크커피 가산 플래그십 스토어 
호세가 
마벨리에 시흥점 
문정곱창 
우마왕 
지리산삼계탕 
1  of  [  1  ] 
춘천옥 
김사부샤브샤브 
동남집 2호점 
황제짬뽕양꼬치 
파티엘하우스 
은행골 시흥분점 
진영면옥 
니하우 
우시장3호집 
인생양대창 
스시쥬베이 
전주식당 
가산어시장 본점 
이화룡불짬뽕 
스미다 
2  of  [  1  ] 
왕게수산 
맥도날드 서울시흥DT점 
동남집 
말뚝곱창 6호점 
탐나종합어시장 가산점 
휴스시 
아웃백스테이크하우스 가산점 
논빼미돼지볶음 
말뚝곱창 5호점 
여신족발 
딥 
풍년갈비 
조개가득 가산본점 
종로계림 닭도리탕원조 가산디지털단지점 
떡볶이랑걸레만두랑 
3  of  [  1  ] 
스튜디오레이크 
로 
옛날닭꼬치사골부대찌개 
도넛인사이드도넛 
암소한마리 본점 
냥카페묘연 
초가집부뚜막청국장 독산점 
스타벅스 금천독산점 
삼은식당 
제주탐하리 
화로구이 
금천가든 
만두라 
이화순대국 
통영해물가 가산2호점 
4  of  [  1  ] 
이유 본점 
우리집 순대국 
스시초이 
청보리한우 
삼숙이갈비 가산디지털본점 
텐마루 가산점 
버거킹 가산디지털점 
키주파크 독산점 
본가집가마솥설렁탕 금천구청본점 
천지회관 
미요리포차 
실비순대국 
고기싸롱 가산점 
생고기한상 
서대문김치찜 
5  of  [  1  ] 
다도해수산 
총각네부추곱창 독산동본점 
스타벅스 금천시흥점 
뽀로로파크 롯데빅마켓 금천점 
스타벅스 독산사거리점 
태영생막창 독산점 
실크로드중화요리 
카페로드 
맥도날드 가산비지니스센터점 
협진식당 5호점 
신미경정통춘천닭갈비 독산직영점 
스타벅스 가산디지털점 
팔람까오 
종가대박집 금천점 
야시장 
1  of  [  2  ] 
영향 
왕래성 
우리소양평해장국 본

22it [21:32:58, 3508.93s/it]

End of Page
Arrow is Disabled
End of Crawl
신림춘천집 구로디지털직영점 
월래순교자관 
강촌숯불닭갈비 본점 
화양가옥 구로디지털단지점 
갓파스시 구로점 
컴포 신도림점 
평양냉면 
메이비 
신도림참족발 신도림점 
스시메이진 구로점 
봉고기 신도림본점 
최우영스시 
진심갈비 
송림가 
속초오징어어시장 구로디지털점 
1  of  [  1  ] 
초가집부뚜막청국장 
한판참숯소갈비 구로디지털점 
어다리횟집 구로점 
바르미스시샤브 현대백화점 디큐브시티점 
이춘복참치 신도림점 
오사카로 
세광양대창 구로디지털점 
새벽집양곱창 신도림점 
오비스트로 
제주도초밥 
에버애프터 
진사향 
밤새먹소 
육장장이 구로디지털점 
실크로드 
2  of  [  1  ] 
소담촌 고척점 
백년불고기 구로디지털점 
더족발 신도림본점 
옛날마차 신관 
닥터로빈 고척점 
속초붉은대게 
서일순대국전문 구로점 
서울안심축산정육식당 신도림점 
대두족발 
오목집 신도림점 
두레한식부페 
갈비명가 
신도림이도식당 
숙이네조개전골 신도림점 
신선회수산 신도림점 
3  of  [  1  ] 
손가네생오리부페 
고바우 
라도맨션 
자금성 
곱창대장 
사계진미숯불닭갈비 구로디지털점 
다원국수 
뒤뜰 
다채 개봉점 
수뿌레닭갈비 신도림점 
말뚝곱창 루프탑점 
라마다서울신도림호텔 카페라라 
동서네낙지 신도림점 
김영태의이자카야마을 
청담이상 구로디지털점 
4  of  [  1  ] 
육식예찬 
택이네조개전골 구로디지털직영점 
풍천참숯불즉석장어구이 
라마노피자 
속초붉은대게 
미식가양꼬치 
브라운핸즈 디지털단지구로점 
이천성모메존칼국수 본점 
방앗간정육식당 
제면청년 
청담이상 신도림점 
라꾸긴 
스시웨이 신도림점 
바다쭈꾸미 개봉점 
쭉심 오류동점 
5  of  [  1  ] 
진참치 
문어부인삼교비 신도림점 
니와참치 
히어로스터 신도림점 
품격사시미 
왕뼈해장국 고척점 
춘자싸롱 
